In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

# (a) Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/ Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\ %29 . The dataset contains 7 folders that represent seven types of activities. In each folder, there are multiple files each of which represents an instant of a human performing an activity.1 Each file containis 6 time series collected from activitiesof the same person, which are called avg rss12, var rss12, avg rss13, var rss13, vg rss23, and ar rss23. There are 88 instances in the dataset, each of which con- tains 6 time series and each time series has 480 consecutive values.


> The corresponding data is downloaded and placed in the '../data' Directory 

# (b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

In [6]:
# folders = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']
folders = {
    'bending1': 7,
    'bending2': 6,
    'cycling': 15,
    'lying' : 15,
    'sitting': 15,
    'standing': 15,
    'walking': 15  
}
testing_paths = []
training_paths = []

for activity, _max in folders.items():
    for i in range(1, _max + 1):
        if i <= 2:
            testing_paths.append(f'../data/{activity}/dataset{i}.csv')    
        elif i == 3 and activity not in ['bending1', 'bending2']:
            testing_paths.append(f'../data/{activity}/dataset{i}.csv')   
        else:
            training_paths.append(f'../data/{activity}/dataset{i}.csv')
            
columns = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
testing_dfs = [pd.read_csv(path, skiprows = 4) for path in testing_paths]
training_dfs = [pd.read_csv(path, skiprows = 4) for path in training_paths]
    
train_df = pd.concat(training_dfs)
test_df = pd.concat(testing_dfs)
print(train_df.shape)
print(test_df.shape)

(33119, 7)
(9120, 7)


In [7]:
train_df.head(10)
test_

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,39.25,0.43,22.75,0.43,33.75,1.3
1,250,39.25,0.43,23.00,0.00,33.00,0.0
2,500,39.25,0.43,23.25,0.43,33.00,0.0
3,750,39.50,0.50,23.00,0.71,33.00,0.0
4,1000,39.50,0.50,24.00,0.00,33.00,0.0
5,1250,39.25,0.43,24.00,0.00,33.00,0.0
6,1500,39.25,0.43,24.00,0.00,33.00,0.0
7,1750,39.00,0.00,23.75,0.43,33.00,0.0
8,2000,39.50,0.50,24.00,0.00,33.00,0.0
9,2250,39.50,0.50,23.00,0.00,33.00,0.0
